## Add utils as a module and import

In [1]:
import sys
    
try:
    from utils import data_loading
except:
    sys.path.append('../utils')
    sys.path.append('../')
    from utils import data_loading

# Loading libraries

In [2]:
import os
import numpy as np
from utils import model
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
from nilearn import datasets
from nilearn import plotting
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as corr
from sklearn.decomposition import PCA

# Loading Data

In [3]:
#loading data for subject 1, first 10 images
lh, rh, images  = data_loading.load_subject_data(1, 0, 1000)

print(lh.shape)
print(rh.shape)
print(images[0].shape)

(1000, 19004)
(1000, 20544)
(425, 425, 3)


In [4]:
#concat lh and rh
brain = np.concatenate((lh, rh), axis=1)

### Load dataset for one-head

In [5]:
dataset = data_loading.CustomDataset(images_list= images, outputs_list = brain, transform=transforms.ToTensor(), PCA = PCA(n_components = 100))
print('\nDataset made up of ', len(dataset), 'pairs of data\n--------')
print('Shape of 1st element in pair:', dataset[0][0].shape)
print('Shape of 2nd element in pair:', dataset[0][1].shape)

#Create a train and validation subset of variable dataset with torch
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
#put train dataset into a loader with 2 batches and put test data in val loader
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True)

       
Initialize CustomDataset 
--------
Number of samples:  1000
Transform:  ToTensor()
PCA:  PCA(n_components=100)

Data loaded
-------
Data:  1000 *  (<PIL.Image.Image image mode=RGB size=425x425 at 0x7FCDC2C25A90>, array([-0.8617882 , -0.20318632, -0.62639767, ..., -0.41889378,
       -0.60231453, -0.67537224], dtype=float32))
Output_concat:  1000 * 39548 :  [-0.8617882  -0.20318632 -0.62639767 ... -0.41889378 -0.60231453
 -0.67537224]

Dataset made up of  1000 pairs of data
--------
Shape of 1st element in pair: torch.Size([3, 425, 425])
Shape of 2nd element in pair: torch.Size([100])


### Dataset for 2head

In [8]:
dataset2 = CustomDataset1(images_list= images, outputs_list1 = lh, outputs_list2 = rh, transform=transforms.ToTensor(), PCA1 = PCA(n_components = 100), PCA2 = PCA(n_components = 100))
print('\nDataset made up of ', len(dataset2), 'truples of data\n--------')
print('Shape of 1st element:', dataset2[0][0].shape)
print('Shape of 2nd element:', dataset2[0][1].shape)
print('Shape of 3nd element:', dataset2[0][2].shape)

#Create a train and validation subset of variable dataset with torch
train_size = int(0.8 * len(dataset2))
test_size = len(dataset2) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset2, [train_size, test_size])
#put train dataset into a loader with 2 batches and put test data in val loader
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True)

Number of samples: 1000
Output_concat1: 19004
Output_concat2: 20544

Dataset made up of  1000 truples of data
--------
Shape of 1st element: torch.Size([3, 425, 425])
Shape of 2nd element: torch.Size([100])
Shape of 3nd element: torch.Size([100])


In [9]:
reg_model = model.get_pretrained_regression_model(100)
reg_model2 = model.ResNet2HeadModel(100)
trainer = model.Trainer()
optimizer = torch.optim.Adam
loss = torch.nn.MSELoss()
trainer.compile(reg_model2, optimizer, learning_rate=0.0001, loss_fn=loss)

In [10]:
reg_model2

ResNet2HeadModel(
  (pretrained_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [11]:
trainer.fit_dual_head(num_epochs= 3, train_loader=train_loader, val_loader=val_loader)

batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
Epoch [1/3], Training Loss: 147.24491653442382
Validation Loss: 149.16273788452148
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
batch
Epoch [2/3], Training Loss: 144.9409878540039


KeyboardInterrupt: 

In [22]:
for inputs in train_loader:
    print(inputs[0][0].shape)
    print(inputs[1].shape)
    # print(inputs[2].shape)
    print(inputs[1])
    break

torch.Size([3, 425, 425])
torch.Size([16, 100])
tensor([[-6.9004e+01,  5.2036e+01, -1.3860e+01,  ..., -1.1964e+00,
         -5.9054e+00, -3.0465e+00],
        [ 2.8790e+01, -6.7930e+01,  5.3093e+01,  ...,  2.5009e+00,
         -1.1363e+00, -9.1270e+00],
        [-1.0563e+01,  3.1635e+01, -6.0819e+01,  ..., -2.8365e+00,
         -1.0647e-02, -1.1718e+00],
        ...,
        [ 4.8215e+01,  1.5110e+01,  5.9402e+01,  ..., -1.2612e+01,
         -4.5678e+00, -9.4691e+00],
        [ 1.1411e+01, -2.3718e+01, -3.3905e+01,  ..., -1.1347e+01,
          9.9543e+00, -1.3719e+00],
        [-3.7915e+01,  1.9135e+01, -2.4873e+01,  ..., -4.2263e+00,
         -4.8566e-01, -1.6125e+00]])


In [14]:
trainer.save('initial_model.pt')

Model and training history saved to initial_model.pt


In [26]:
train_dataset[0][2].shape

IndexError: tuple index out of range

In [11]:
trainer  = model.Trainer()
trainer.compile(reg_model, optimizer, learning_rate=0.0001, loss_fn=loss)

In [28]:
trainer.load('initial_model.pt')

Model and training history loaded from initial_model.pt


In [29]:
trainer.history

{'train_loss': [170.70301513671876, 167.72671508789062],
 'val_loss': [183.5020439147949, 183.56376399993897]}

In [7]:
from sklearn.decomposition import PCA
import torch
from torch.utils.data import Dataset
from PIL import Image

class CustomDataset1(Dataset):
    def __init__(self, images_list, outputs_list1, outputs_list2, transform=None, PCA1=None, PCA2=None):
        self.num_samples = len(images_list)
        print('Number of samples:', self.num_samples)
        self.transform = transform
        self.PCA1 = PCA1
        self.PCA2 = PCA2
        self.data, self.output1, self.output2 = self.load_data(images_list, outputs_list1, outputs_list2)

    def load_data(self, images_list, outputs_list1, outputs_list2):
        data = []
        output_concat1 = []
        output_concat2 = []

        for i in range(self.num_samples):
            # Load image from the given list
            image = Image.fromarray(images_list[i])

            # Load output arrays from the given lists
            output1 = outputs_list1[i]
            output2 = outputs_list2[i]

            data.append((image, output1, output2))
            output_concat1.append(output1)
            output_concat2.append(output2)

        if self.PCA1:
            self.PCA1.fit(output_concat1)

        if self.PCA2:
            self.PCA2.fit(output_concat2)

        print('Output_concat1:', len(output_concat1[0]))
        print('Output_concat2:', len(output_concat2[0]))
        return data, output_concat1, output_concat2

    def give_output(self):
        return self.output1, self.output2

    def get_PCA1(self):
        return self.PCA1

    def get_PCA2(self):
        return self.PCA2

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        image, output1, output2 = self.data[idx]

        if self.transform:
            image = self.transform(image)

        if self.PCA1:
            output1 = self.PCA1.transform(output1.reshape(1, -1))

        if self.PCA2:
            output2 = self.PCA2.transform(output2.reshape(1, -1))

        return image, torch.FloatTensor(output1[0]), torch.FloatTensor(output2[0])

In [8]:
dataset = CustomDataset1(images_list= images, outputs_list1= brain_separate[0], outputs_list2 = brain_separate[1], transform=transforms.ToTensor(), PCA1 = PCA(n_components = 100), PCA2 = PCA(n_components = 100))